In [1]:
from __future__ import absolute_import, division, print_function

import argparse
from datetime import datetime
import numpy as np
import os

from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard
import numpy as np
from sklearn.metrics import roc_auc_score

from data_handler import DataHandler
from models import create_grud_model, load_grud_model
from nn_utils.callbacks import ModelCheckpointwithBestWeights

import warnings
warnings.filterwarnings("ignore")


Using TensorFlow backend.


In [2]:
# set GPU usage for tensorflow backend
if K.backend() == 'tensorflow':
    import tensorflow as tf
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = .1
    config.gpu_options.allow_growth = True
    K.set_session(tf.Session(config=config))

In [3]:
# parse arguments
## general
arg_parser = argparse.ArgumentParser()
arg_parser.add_argument('--working_path', default='.')

## data
arg_parser.add_argument('dataset_name', default='mimic3',
                        help='The data files should be saved in [working_path]/data/[dataset_name] directory.')
arg_parser.add_argument('label_name', default='mortality')
arg_parser.add_argument('--max_timesteps', type=int, default=200, 
                        help='Time series of at most # time steps are used. Default: 200.')
arg_parser.add_argument('--max_timestamp', type=int, default=48*60*60,
                        help='Time series of at most # seconds are used. Default: 48 (hours).')

## model
arg_parser.add_argument('--recurrent_dim', type=lambda x: x and [int(xx) for xx in x.split(',')] or [], default='64')
arg_parser.add_argument('--hidden_dim', type=lambda x: x and [int(xx) for xx in x.split(',')] or [], default='64')
arg_parser.add_argument('--model', default='GRUD', choices=['GRUD', 'GRUforward', 'GRU0', 'GRUsimple'])
arg_parser.add_argument('--use_bidirectional_rnn', default=False)
                           
## training
arg_parser.add_argument('--pretrained_model_file', default=None,
                        help='If pre-trained model is provided, training will be skipped.') # e.g., [model_name]_[i_fold].h5
arg_parser.add_argument('--epochs', type=int, default=100)
arg_parser.add_argument('--early_stopping_patience', type=int, default=10)
arg_parser.add_argument('--batch_size', type=int, default=32)


## set the actual arguments if running in notebook
if not (__name__ == '__main__' and '__file__' in globals()):
    ARGS = arg_parser.parse_args([
        'sample',
        'taskname',
        '--model', 'GRUD',
        '--hidden_dim', '',
        '--epochs', '100'
    ])
else:
    ARGS = arg_parser.parse_args()

print('Arguments:', ARGS)

Arguments: Namespace(batch_size=32, dataset_name='sample', early_stopping_patience=10, epochs=100, hidden_dim=[], label_name='taskname', max_timestamp=172800, max_timesteps=200, model='GRUD', pretrained_model_file=None, recurrent_dim=[64], use_bidirectional_rnn=False, working_path='.')


In [4]:
# get dataset
dataset = DataHandler(
    data_path=os.path.join(ARGS.working_path, 'data', ARGS.dataset_name), 
    label_name=ARGS.label_name,
    max_steps=ARGS.max_timesteps,
    max_timestamp=ARGS.max_timestamp
)

In [5]:
# k-fold cross-validation
pred_y_list_all = []
auc_score_list_all = []

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S_%f')
print('Timestamp: {}'.format(timestamp))

for i_fold in range(dataset.folds):
    print('{}-th fold...'.format(i_fold))

    # Load or train the model.
    if ARGS.pretrained_model_file is not None:
        model = load_grud_model(os.path.join(ARGS.working_path, 
                                             ARGS.pretrained_model_file.format(i_fold=i_fold)))
    else:
        model = create_grud_model(input_dim=dataset.input_dim,
                                  output_dim=dataset.output_dim,
                                  output_activation=dataset.output_activation,
                                  recurrent_dim=ARGS.recurrent_dim,
                                  hidden_dim=ARGS.hidden_dim,
                                  predefined_model=ARGS.model,
                                  use_bidirectional_rnn=ARGS.use_bidirectional_rnn
                                 )
        if i_fold == 0:
            model.summary()
        model.compile(optimizer='adam', loss=dataset.loss_function)
        model.fit_generator(
            generator=dataset.training_generator(i_fold, batch_size=ARGS.batch_size),
            steps_per_epoch=dataset.training_steps(i_fold, batch_size=ARGS.batch_size),
            epochs=ARGS.epochs,
            verbose=1,
            validation_data=dataset.validation_generator(i_fold, batch_size=ARGS.batch_size),
            validation_steps=dataset.validation_steps(i_fold, batch_size=ARGS.batch_size),
            callbacks=[
                EarlyStopping(patience=ARGS.early_stopping_patience),
                ModelCheckpointwithBestWeights(
                    file_dir=os.path.join(ARGS.working_path, 'model', timestamp + '_' + str(i_fold))
                ),
                TensorBoard(
                    log_dir=os.path.join(ARGS.working_path, 'tb_logs', timestamp + '_' + str(i_fold))
                )
            ]
            )
        model.save(os.path.join(ARGS.working_path, 'model', 
                                timestamp + '_' + str(i_fold), 'model.h5'))

    # Evaluate the model
    true_y_list = [
        dataset.training_y(i_fold), dataset.validation_y(i_fold), dataset.testing_y(i_fold)
    ]
    pred_y_list = [
        model.predict_generator(dataset.training_generator_x(i_fold, batch_size=ARGS.batch_size),
                                steps=dataset.training_steps(i_fold, batch_size=ARGS.batch_size)),
        model.predict_generator(dataset.validation_generator_x(i_fold, batch_size=ARGS.batch_size),
                                steps=dataset.validation_steps(i_fold, batch_size=ARGS.batch_size)),
        model.predict_generator(dataset.testing_generator_x(i_fold, batch_size=ARGS.batch_size),
                                steps=dataset.testing_steps(i_fold, batch_size=ARGS.batch_size)),
    ]
    auc_score_list = [roc_auc_score(ty, py) for ty, py in zip(true_y_list, pred_y_list)] # [3, n_task]
    print('AUC score of this fold: {}'.format(auc_score_list))
    pred_y_list_all.append(pred_y_list)
    auc_score_list_all.append(auc_score_list)

print('Finished!', '='*20)
auc_score_list_all = np.stack(auc_score_list_all, axis=0)
print('Mean AUC score: {}; Std AUC score: {}'.format(
    np.mean(auc_score_list_all, axis=0),
    np.std(auc_score_list_all, axis=0)))

result_path = os.path.join(ARGS.working_path, 'results', timestamp)
if not os.path.exists(result_path):
    os.makedirs(result_path)
np.savez_compressed(os.path.join(result_path, 'predictions.npz'),
                    pred_y_list_all=pred_y_list_all)
np.savez_compressed(os.path.join(result_path, 'auroc_score.npz'),
                    auc_score_list_all=auc_score_list_all)


Timestamp: 20230505_184652_394531
0-th fold...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 7)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 7)      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None, 1)      0                                            
__________________________________________________________________________________________________
external_masking_1 (ExternalMas (None, None, 7)      0           input_1[0][0]                    
                                                              

2/2 [==============================] - 0s 52ms/step - loss: 0.4718 - val_loss: 0.4285
Epoch 54/100
2/2 [==============================] - 0s 39ms/step - loss: 0.5026 - val_loss: 0.4262
Epoch 55/100
2/2 [==============================] - 0s 44ms/step - loss: 0.4908 - val_loss: 0.4240
Epoch 56/100
2/2 [==============================] - 0s 46ms/step - loss: 0.4705 - val_loss: 0.4217
Epoch 57/100
2/2 [==============================] - 0s 49ms/step - loss: 0.4692 - val_loss: 0.4200
Epoch 58/100
2/2 [==============================] - 0s 44ms/step - loss: 0.4847 - val_loss: 0.4181
Epoch 59/100
2/2 [==============================] - 0s 40ms/step - loss: 0.4644 - val_loss: 0.4162
Epoch 60/100
2/2 [==============================] - 0s 43ms/step - loss: 0.4771 - val_loss: 0.4141
Epoch 61/100
2/2 [==============================] - 0s 38ms/step - loss: 0.4755 - val_loss: 0.4122
Epoch 62/100
2/2 [==============================] - 0s 48ms/step - loss: 0.4309 - val_loss: 0.4104
Epoch 63/100
2/2 [=====

2/2 [==============================] - 0s 40ms/step - loss: 0.4929 - val_loss: 0.5270
Epoch 35/100
2/2 [==============================] - 0s 39ms/step - loss: 0.4893 - val_loss: 0.5218
Epoch 36/100
2/2 [==============================] - 0s 48ms/step - loss: 0.5035 - val_loss: 0.5170
Epoch 37/100
2/2 [==============================] - 0s 41ms/step - loss: 0.4742 - val_loss: 0.5125
Epoch 38/100
2/2 [==============================] - 0s 44ms/step - loss: 0.4749 - val_loss: 0.5083
Epoch 39/100
2/2 [==============================] - 0s 42ms/step - loss: 0.4852 - val_loss: 0.5044
Epoch 40/100
2/2 [==============================] - 0s 37ms/step - loss: 0.4795 - val_loss: 0.5009
Epoch 41/100
2/2 [==============================] - 0s 40ms/step - loss: 0.4721 - val_loss: 0.4977
Epoch 42/100
2/2 [==============================] - 0s 52ms/step - loss: 0.4660 - val_loss: 0.4944
Epoch 43/100
2/2 [==============================] - 0s 40ms/step - loss: 0.4965 - val_loss: 0.4912
Epoch 44/100
2/2 [=====

2/2 [==============================] - 0s 40ms/step - loss: 0.5172 - val_loss: 0.6767
Epoch 16/100
2/2 [==============================] - 0s 48ms/step - loss: 0.4984 - val_loss: 0.6752
Epoch 17/100
2/2 [==============================] - 0s 44ms/step - loss: 0.5215 - val_loss: 0.6740
Epoch 18/100
2/2 [==============================] - 0s 40ms/step - loss: 0.5250 - val_loss: 0.6720
Epoch 19/100
2/2 [==============================] - 0s 43ms/step - loss: 0.5094 - val_loss: 0.6701
Epoch 20/100
2/2 [==============================] - 0s 41ms/step - loss: 0.5259 - val_loss: 0.6691
Epoch 21/100
2/2 [==============================] - 0s 45ms/step - loss: 0.5129 - val_loss: 0.6678
Epoch 22/100
2/2 [==============================] - 0s 44ms/step - loss: 0.5117 - val_loss: 0.6657
Epoch 23/100
2/2 [==============================] - 0s 44ms/step - loss: 0.4950 - val_loss: 0.6633
Epoch 24/100
2/2 [==============================] - 0s 47ms/step - loss: 0.4737 - val_loss: 0.6604
Epoch 25/100
2/2 [=====

Epoch 98/100
2/2 [==============================] - 0s 48ms/step - loss: 0.3228 - val_loss: 0.4885
Epoch 99/100
2/2 [==============================] - 0s 44ms/step - loss: 0.3179 - val_loss: 0.4880
Epoch 100/100
2/2 [==============================] - 0s 38ms/step - loss: 0.3350 - val_loss: 0.4874
..............................AUC score of this fold: [0.8761281799476244, 0.7290140415140415, 0.7393162393162394]
3-th fold...
Epoch 1/100
2/2 [==============================] - 1s 639ms/step - loss: 0.8968 - val_loss: 0.8361
Epoch 2/100
2/2 [==============================] - 0s 36ms/step - loss: 0.8781 - val_loss: 0.7956
Epoch 3/100
2/2 [==============================] - 0s 45ms/step - loss: 0.7849 - val_loss: 0.7587
Epoch 4/100
2/2 [==============================] - 0s 39ms/step - loss: 0.7723 - val_loss: 0.7249
Epoch 5/100
2/2 [==============================] - 0s 44ms/step - loss: 0.7496 - val_loss: 0.6939
Epoch 6/100
2/2 [==============================] - 0s 46ms/step - loss: 0.7115 - va

2/2 [==============================] - 0s 49ms/step - loss: 0.4043 - val_loss: 0.4187
Epoch 80/100
2/2 [==============================] - 0s 38ms/step - loss: 0.4030 - val_loss: 0.4172
Epoch 81/100
2/2 [==============================] - 0s 43ms/step - loss: 0.3815 - val_loss: 0.4157
Epoch 82/100
2/2 [==============================] - 0s 40ms/step - loss: 0.3623 - val_loss: 0.4146
Epoch 83/100
2/2 [==============================] - 0s 55ms/step - loss: 0.3979 - val_loss: 0.4136
Epoch 84/100
2/2 [==============================] - 0s 40ms/step - loss: 0.3802 - val_loss: 0.4125
Epoch 85/100
2/2 [==============================] - 0s 63ms/step - loss: 0.3863 - val_loss: 0.4117
Epoch 86/100
2/2 [==============================] - 0s 40ms/step - loss: 0.3718 - val_loss: 0.4108
Epoch 87/100
2/2 [==============================] - 0s 40ms/step - loss: 0.3860 - val_loss: 0.4098
Epoch 88/100
2/2 [==============================] - 0s 58ms/step - loss: 0.3691 - val_loss: 0.4093
Epoch 89/100
2/2 [=====

2/2 [==============================] - 0s 42ms/step - loss: 0.4566 - val_loss: 0.4106
Epoch 61/100
2/2 [==============================] - 0s 41ms/step - loss: 0.4293 - val_loss: 0.4087
Epoch 62/100
2/2 [==============================] - 0s 48ms/step - loss: 0.4560 - val_loss: 0.4068
Epoch 63/100
2/2 [==============================] - 0s 41ms/step - loss: 0.4473 - val_loss: 0.4050
Epoch 64/100
2/2 [==============================] - 0s 42ms/step - loss: 0.4611 - val_loss: 0.4033
Epoch 65/100
2/2 [==============================] - 0s 36ms/step - loss: 0.4615 - val_loss: 0.4017
Epoch 66/100
2/2 [==============================] - 0s 41ms/step - loss: 0.4341 - val_loss: 0.3999
Epoch 67/100
2/2 [==============================] - 0s 40ms/step - loss: 0.4318 - val_loss: 0.3981
Epoch 68/100
2/2 [==============================] - 0s 62ms/step - loss: 0.4564 - val_loss: 0.3962
Epoch 69/100
2/2 [==============================] - 0s 49ms/step - loss: 0.4533 - val_loss: 0.3943
Epoch 70/100
2/2 [=====